In [4]:
import os
import json
from PIL import Image

input_dir = "../data/annotated"
output_dir = "../data/labels"
image_dir = "../data/tiles"
class_name = "ship"  # change if needed

# Make sure output folder exists
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith(".json"):
        json_path = os.path.join(input_dir, filename)
        with open(json_path) as f:
            data = json.load(f)

        img_path = os.path.join(image_dir, data["imagePath"])
        with Image.open(img_path) as img:
            img_w, img_h = img.size

        label_path = os.path.join(output_dir, filename.replace(".json", ".txt"))
        with open(label_path, "w") as out:
            for shape in data["shapes"]:
                if shape["shape_type"] != "rectangle":
                    continue  # or handle polygons if you labeled them

                points = shape["points"]
                x1, y1 = points[0]
                x2, y2 = points[1]

                # Normalize to YOLO format: x_center, y_center, width, height
                x_center = ((x1 + x2) / 2) / img_w
                y_center = ((y1 + y2) / 2) / img_h
                width = abs(x2 - x1) / img_w
                height = abs(y2 - y1) / img_h

                out.write(f"0 {x_center} {y_center} {width} {height}\n")  # class 0 = "ship"
